# K-visitation

In this paper, we introduce the novel *K-Visitation* framework, which quantifies the minimal number of visitations, K, required for an individual to satisfy all essential amenity exposure. This framework operates under two orderings: (i) K-Frequency ($K_{Freq}$), in which visitations are selected by empirical visitation frequency, characterising their recurrent behaviour, and (ii) K-Distance ($K_{Dist}$), in which visitations are selected by proximity to home, representing their proximate baseline scenario.

This notebook provides an operationalised sample of this framework using an anonymised dataset, which does not contain any identifable spatial or user information.

### Load libraries and data

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

# Setup project root path
project_root = Path.cwd()
if 'notebooks' in str(project_root):
    project_root = project_root.parent

# Add src to path
src_path = str(project_root / 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Set data path
data_dir = project_root / 'data'

In [ ]:
# Load the anonymised stay location data sample

# For privacy reasons, we excluded spatial informations from this dataset, and user_ids were hashed. 
# Visitation attributes needed have been added in previous steps, which includes:
# 1. Euclidean distance from visitation to home location;
# 2. Frequency of visitation (within the same H3 hexagon);
# 3. Number of POIs in each category within 400m buffer of each visitation.
# Therefore, it is not possible to identify the exact locations of stay points from the given sampled data.

data_path = data_dir / 'sample_anonymised_data.csv'
stay_locations = pd.read_csv(data_path)
stay_locations.head()

### Framework operation

In [ ]:
from k_visitation import calculate_both_k_places

In [ ]:
amenity_list = [
    'CIVIC_RELIGION', 'CULTURE', 'DINING', 'EDUCATION', 'FITNESS', 'GROCERIES', 'HEALTHCARE', 'RETAIL', 'SERVICE', 'TRANSPORT'
    ]

smallest_values = np.ones(len(amenity_list), dtype=int)


# Apply the K-places calculation
places_k = calculate_both_k_places(stay_locations, amenity_list, smallest_values)

places_k.head()

In [ ]:
# Check per-user K-freq and K-dist places (deduplicating stay_gid10 per user)
user_k_counts = (
    places_k.groupby("user_id")
    .agg(
        k_freq_places=("k_freq", lambda s: (s == 1).sum()),
        k_dist_places=("k_dist", lambda s: (s == 1).sum()),
    )
    .reset_index()
)

print(user_k_counts.head())

### Alignment between recurrent and proximate mobility

In [ ]:
from k_visitation import calculate_qk_alignment

In [ ]:
# Calculate qK alignment using the no-work variants
user_qk = calculate_qk_alignment(
    places_df=places_k,
    user_col='user_id', 
    k_freq_col='k_freq',  # Using no-work version
    k_dist_col='k_dist'   # Using no-work version
)

In [ ]:
user_qk.head()